In [1]:
from bs4 import BeautifulSoup

from getpass import getpass
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import pandas as pd
import requests
import time

In [13]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://nomadlist.com/")


In [152]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

In [14]:
screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
i = 1
time.sleep(2)  # Allow 2 seconds for the web page to open
scroll_pause_time = 1
while True:
    # scroll one screen height each time
    driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
    i += 1
    time.sleep(scroll_pause_time)
    # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
    scroll_height = driver.execute_script("return document.body.scrollHeight;")  
    # Break the loop when the height we need to scroll to is larger than the total scroll height
    if (screen_height) * i > scroll_height:
        break

In [160]:
### City Name
city = list(map(lambda name: name.text, driver.find_elements_by_class_name('itemName')))

In [159]:
### Country
country = list(map(lambda cont: cont.text, driver.find_elements_by_class_name('itemSub')))

In [158]:
### Ranking
rank = list(map(lambda rk: rk.text, driver.find_elements_by_class_name('rank')))

In [157]:
### Cost of living
cost = list(map(lambda ct: ct.text, driver.find_elements_by_class_name('price')))

In [19]:
### Description
desc = list(map(lambda de: de.text, driver.find_elements_by_class_name('description')))

In [20]:
desc

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',


In [161]:
### update lists 
### delete 2 position from city
city.remove('') 
city.remove('Visit Asia')

### delete from country
country.remove('')

### delete from cost
cost.remove('')
del cost[1348:]

ValueError: list.remove(x): x not in list

In [162]:
### Check Lengths
print('cost ', len(cost))
print('rank ', len(rank))
print('country ', len(country))
print('city ', len(city))

cost  1348
rank  1348
country  1348
city  1348


In [163]:
### Create DF
nomadlist = pd.DataFrame({
        'Rank': rank,
        'City': city, 
        'Country': country, 
        'Cost of Living': cost, 
    })

In [172]:
### Save to pickle
#nomadlist.to_pickle('NomadList.pickle')

In [2]:
#Read pickle
nomadlist = pd.read_pickle(r'C:\Users\frans\Documents\GitHub\Project-4_WebScraping\NomadList.pickle')
nomadlist

,Rank,City,Country,Cost of Living
0,1,Lisbon,Portugal,"$3,464"
1,2,Chiang Mai,Thailand,"$1,315"
2,3,"Canggu, Bali",Indonesia,"$2,013"
3,4,Bangkok,Thailand,"$2,090"
4,5,Madeira,Portugal,"$2,085"
...,...,...,...,...
1343,1344,Dar es Salaam,Tanzania,"$1,746"
1344,1345,Saint Petersburg,Russia,"$2,384"
1345,1346,Yangon,Myanmar,"$1,330"
1346,1347,Baghdad,Iraq,"$1,904"


In [72]:
### TO DO

# 1 - Fix/ask cristina about extractind "description"

## 2 -Edit Nomadlist dtypes

In [73]:
nomadlist

,Rank,City,Country,Cost of Living
0,1,Lisbon,Portugal,"$3,464"
1,2,Chiang Mai,Thailand,"$1,315"
2,3,"Canggu, Bali",Indonesia,"$2,013"
3,4,Bangkok,Thailand,"$2,090"
4,5,Madeira,Portugal,"$2,085"
...,...,...,...,...
1343,1344,Dar es Salaam,Tanzania,"$1,746"
1344,1345,Saint Petersburg,Russia,"$2,384"
1345,1346,Yangon,Myanmar,"$1,330"
1346,1347,Baghdad,Iraq,"$1,904"


In [3]:
nomadlist['Cost of Living'] = nomadlist['Cost of Living'].str.strip('$')
nomadlist['Cost of Living'] = nomadlist['Cost of Living'].str.replace(',','')
nomadlist['Cost of Living'] = nomadlist['Cost of Living'].astype(int)

In [85]:
#Save to pickle
#nomadlist.to_pickle('nomad.pickle')

## 3 - Scrap ranking from secondary page & pass the column 'city' as url 

In [4]:
### Create df to pass city names as page url
nameurl = nomadlist[['City','Rank']]
citypage = nameurl['City'].str.split(',', expand = True)
citypage[0] = citypage[0].str.lower()
citypage[0] = citypage[0].str.replace(' ','-')
citypage.rename(columns = {0:'city'}, inplace=True)

In [8]:
citypage

,city,1
0,lisbon,None
1,chiang-mai,None
2,canggu,Bali
3,bangkok,None
4,madeira,None
...,...,...
1343,dar-es-salaam,None
1344,saint-petersburg,None
1345,yangon,None
1346,baghdad,None


In [5]:
### Scrap multiple pages 
citylist = citypage.city.tolist()

def get_url(page):
    return 'https://nomadlist.com/{}'.format(page)

url_list = list(map(get_url,citylist))
url_list

['https://nomadlist.com/lisbon',
 'https://nomadlist.com/chiang-mai',
 'https://nomadlist.com/canggu',
 'https://nomadlist.com/bangkok',
 'https://nomadlist.com/madeira',
 'https://nomadlist.com/buenos-aires',
 'https://nomadlist.com/mexico-city',
 'https://nomadlist.com/timisoara',
 'https://nomadlist.com/berlin',
 'https://nomadlist.com/bengaluru',
 'https://nomadlist.com/gran-canaria',
 'https://nomadlist.com/cape-town',
 'https://nomadlist.com/medellín',
 'https://nomadlist.com/penang',
 'https://nomadlist.com/portimão',
 'https://nomadlist.com/lagos',
 'https://nomadlist.com/fuerteventura',
 'https://nomadlist.com/ho-chi-minh-city',
 'https://nomadlist.com/porto',
 'https://nomadlist.com/kuala-lumpur',
 'https://nomadlist.com/tenerife',
 'https://nomadlist.com/ko-pha-ngan',
 'https://nomadlist.com/brasov',
 'https://nomadlist.com/nairobi',
 'https://nomadlist.com/kathmandu',
 'https://nomadlist.com/istanbul',
 'https://nomadlist.com/ubud',
 'https://nomadlist.com/varna',
 'https:/

In [36]:
import re
import numpy as np
import random
### Web scrapping rank
driver = webdriver.Chrome(ChromeDriverManager().install())
def all_process(url):
    driver.get(url)
    x = random.randint(1,2)
    time.sleep(x)
    rating01 = list(map(lambda joke: joke.text, driver.find_elements_by_class_name('details')))
    result = re.findall('Rank #([0-9]+)', rating01[0]) if len(rating01)>0 else 'empty'
    print(x,' seconds of time used for getting ', result)
    return result

final = list(map(all_process,url_list[:100]))
driver.close()
final

2  seconds of time used for getting  ['1']
2  seconds of time used for getting  ['3']
1  seconds of time used for getting  ['4']
2  seconds of time used for getting  ['2']
2  seconds of time used for getting  ['5']
2  seconds of time used for getting  ['6']
1  seconds of time used for getting  ['7']
2  seconds of time used for getting  ['8']
2  seconds of time used for getting  ['9']
1  seconds of time used for getting  ['12']
2  seconds of time used for getting  ['11']
2  seconds of time used for getting  ['10']
1  seconds of time used for getting  empty
2  seconds of time used for getting  ['14']
1  seconds of time used for getting  empty
1  seconds of time used for getting  ['180']
1  seconds of time used for getting  ['18']
2  seconds of time used for getting  ['25']
1  seconds of time used for getting  ['15']
2  seconds of time used for getting  ['21']
1  seconds of time used for getting  ['19']
1  seconds of time used for getting  ['22']
2  seconds of time used for getting  ['23'

In [13]:
import re
import numpy as np
import random
### into df

def all_process(url):
    html = requests.get(url)
    print(html)
    x = random.randint(10,20)
    time.sleep(x)
    soup = BeautifulSoup(html.text, 'html.parser')
    rating01 = list(map(lambda joke: joke.get_text(), soup.find_all(class_ = 'details')))
    result = re.findall('Rank #([0-9]+)', rating01[0]) if len(rating01)>0 else 'empty'
    print(x,' seconds of time used for getting ', result)
    return result

final = list(map(all_process,url_list[:2]))
final

<Response [200]>
19  seconds of time used for getting  ['1']
<Response [503]>
11  seconds of time used for getting  empty
<Response [200]>
14  seconds of time used for getting  ['4']
<Response [200]>
19  seconds of time used for getting  ['2']
<Response [200]>
10  seconds of time used for getting  ['5']
<Response [503]>
12  seconds of time used for getting  empty
<Response [200]>
16  seconds of time used for getting  ['7']
<Response [200]>
20  seconds of time used for getting  ['8']
<Response [200]>
11  seconds of time used for getting  ['9']
<Response [200]>
15  seconds of time used for getting  ['12']


[['1'], 'empty', ['4'], ['2'], ['5'], 'empty', ['7'], ['8'], ['9'], ['12']]

In [68]:
### into df
def all_process(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.text, 'html.parser')
    rating01 = list(map(lambda joke: joke.get_text(), soup.find_all(class_ = 'details')))
    return rating01

final = list(map(all_process,url_list))


In [90]:
final

[['⭐️ Total score4.34/5 (Rank #1)2380 reviews❤️ Liked by members👍86% liked it  👎14% disliked it👍 Quality of life scoreGood👶 Family scoreGood🎒 Community score Great💵 Cost🧐 Pricey: $3,450 / mo📡 Internet🏎 Fast: 28Mbps (avg)😝 FunGreat⛅️ Temperature (now)🥶 A bit cold: 15°C59°F (feels 15°C59°F)💦 Humidity (now)💦 Damp: 88%💨 Air quality (now)🌱 Great: 1 US AQI💨 Air quality (annual)🌱 Great: 31 US AQI👌 SafetyGreat👮\u200d♀️ Lack of crime*Great🤝 Lack of racism*Okay🎓 Education level*Mediocre⚡️ Power gridGreat🌊 Vulnerability to climate changeGood💰 Income level*Low:\t$1,646 / mo🙊 English speaking*Great😤 People density🧘\u200d♀️ Low density: 6k ppl/km² (12x12m)🚶 WalkabilityGreat✌️ Peace (no pol. conflict)Good🚦 Traffic safety*Okay🏥 HospitalsBad😄 Happiness*Okay🍸 NightlifeOkay📶 Free WiFi in cityOkay🖥 Places to work fromGreat❄️ A/C or heatingOkay😁 Friendly to foreignersBad🗯 Freedom of speech*Good👩 Female friendlyOkay🌈 LGBTQ+ friendlyGreat🎅 Startup ScoreOkay',
  "🌍 ContinentEurope📡 Internet speed (avg)28 Mbps